#### Project 1 : data conversion to csv file 
1. Sise2local.ALL_CMDT.0213 file의 A0016 code에서, 3년 국채(BM3)이면서 최근월물(spec의 상품선물_종목정보 데이터[A0016]의 40번인 결제월 번호가 001)인 처음으로 filtering되는 상품code를 찾아야 합니다. 예로, 그 code가 KR4165N30007이면, 변수 product_ID로 저장하고
2. (product_ID & A3016모두 포함) 또는 (product_ID & G7016 모두 포함)하고 있는 row만 순서대로 filtering 하여, 새로운 csv file을 생성하여, 첫 row에는 “시간,현재가,체결수량,매도1호가,매수1호가,최종매도매수,총잔량매도,총잔량매수”를 write하고, 그 아래 row부터, filtering된 A3016 및 G7016이 순서대로 write되는 code를 만들어보자.



In [2]:
# data 받기
import pandas as pd
data = pd.read_csv('CMDT.csv' , 'r' , parse_dates = True , header = None , encoding = 'ISO-8859-1')
pd.set_option('display.max_colwidth', -1) # untruncate the row to better understand the data structure 

In [4]:
# 데이타 구조 파악하기 
a = data[0][1]
print(a[:17], "시간")
print(a[17:22], "code")
print(a[425:428], "bond tpy")
print(a[471:474] , "001")
print(a[35:46], "product ID")

[06:34:45-746774] 시간
A0016 code
BM3 bond tpy
002 001
KR4165N6000 product ID


In [5]:
# A0016 code & 3년 국채(BM3)
# regex 사용해서 product ID 뽑기
# ls has two unique product IDs 
import re
initial_filter = []
p = "A0016.*BM3.{43}001"
for i in range(len(data[0])) : 
    if re.findall(p , data[0][i])!=[] : 
        initial_filter.append(data[0][i][35:46])
ls = list(set(initial_filter))

In [6]:
# filtering (product_ID & A3016) | (product_ID & G7016)
result = pd.DataFrame(columns= ["시간", "현재가", "체결수량", "매도1호가", "매수1호가", "최종매도매수", "총잔량매도", "총잔량매수"])
def sf(s):
    if (s[17:22] == "A3016") and (s[22:33] in ls):
        result.loc[len(result)] =  {"시간" : s[:17], "현재가" : int(s[40:48]), "체결수량" : int(s[48:54]), "매도1호가" : None , "매수1호가" : None,  "최종매도매수" : s[152], "총잔량매도" : None, "총잔량매수" : None}
def tf(s):
    if (s[17:22] == "G7016") and (s[22:33] in ls):
        result.loc[len(result)] =  {"시간" : s[:17], "현재가" : int(s[40:48]), "체결수량" : int(s[48:54]), "매도1호가" :  int(s[243:251]) , "매수1호가" :  int(s[161:169]),  "최종매도매수" : int(s[152]), "총잔량매도" :  int(s[235:242]), "총잔량매수" :  int(s[155:160])}
data[0].apply(sf)
data[0].apply(tf)
result

,시간,현재가,체결수량,매도1호가,매수1호가,최종매도매수,총잔량매도,총잔량매수
0,[09:00:00-023432],10751,1482,None,None,,None,None
1,[09:00:18-013067],10751,1,None,None,2,None,None
2,[09:01:06-567719],10751,45,None,None,2,None,None
3,[09:02:32-516428],10752,3,None,None,2,None,None
4,[09:07:09-057341],10753,68,None,None,1,None,None
5,[09:07:09-122564],10753,4,None,None,1,None,None
6,[09:07:59-690384],10753,84,None,None,2,None,None
7,[09:07:59-721388],10753,3,None,None,1,None,None
8,[09:23:36-552679],10752,52,None,None,2,None,None
9,[09:40:12-972686],10751,65,None,None,1,None,None
